In [1]:
import numpy as np
import json
import linecache
from scipy import sparse

In [2]:
def encode(labels):
    return (labels -100)/300
def decode(enc_labels):
    return (labels*300)+100

In [3]:
#For testing purposes
abatch_size = 5
number_to_train = 80
number_to_val   = 10
number_to_test  = 10
file_name = "features_binned_sparse_50_50_100.txt"
shape_3D = [50, 50, 100]

In [ ]:
# rough detector
abatch_size = 25
number_to_train = 8000
number_to_val   = 1000
number_to_test  = 1000
file_name = "features_binned_sparse_15_15_25.txt"
shape_3D = [15, 15, 25]

In [4]:
# Better detector
abatch_size = 25
number_to_train = 8000
number_to_val   = 1000
number_to_test  = 1000
file_name = "features_binned_sparse_50_50_100.txt"
shape_3D = [50, 50, 100]

In [5]:
def batch_generator_getline(batch_size=25, use_case='train'):
    assert use_case in ['train', 'val', 'test']
        
    if use_case == 'train':
        samples_per_epoch = number_to_train
        shuffle_index = np.arange(0, number_to_train)
    if use_case == 'val':
        samples_per_epoch = number_to_val
        shuffle_index = np.arange(number_to_train, number_to_train + number_to_val)
    if use_case == 'test':
        samples_per_epoch = number_to_test
        shuffle_index = np.arange(number_to_train + number_to_val, number_to_train + number_to_val + number_to_test)
    
    
    
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    
    np.random.shuffle(shuffle_index)
    
    #results_orig = np.load("results2_easy.npy")
    #results_enc = encode(results_orig) 

    while 1:
        index_batch = shuffle_index[batch_size*counter:batch_size*(counter+1)]
        for i, j in enumerate(index_batch):
            js = linecache.getline(file_name, j+1)
            l_js = js.split("\t")
            #print i, l_js
            
            data, indices, indptr, pvr_z_enc = l_js[0], l_js[1], l_js[2], l_js[3]
            #print indices
            indices = np.array(json.loads(indices));  indptr  = np.array(json.loads(indptr));  data    = np.array(json.loads(data)) 
            pvr_z_enc = np.array(json.loads(pvr_z_enc)) 
            sparse2 = sparse.csr_matrix((data, indices, indptr), shape=(shape_3D[0]*shape_3D[1], shape_3D[2]))
            dense2  = np.array(sparse2.todense()).reshape(shape_3D[0], shape_3D[1], shape_3D[2],1)

            if i == 0:
                features_binned = np.array([dense2])
                results_enc = np.array([pvr_z_enc])
            else:
                features_binned = np.append(features_binned, [dense2], axis=0)
                results_enc = np.append(results_enc, [pvr_z_enc])


        features_binned = features_binned.reshape(batch_size,shape_3D[0], shape_3D[1], shape_3D[2],1) 
        counter += 1
        yield(features_binned,results_enc)
        if (counter > number_of_batches):
            np.random.shuffle(shuffle_index)
            counter=0

In [6]:
def batch_generator_readlines(batch_size=25, use_case='train'):
    assert use_case in ['train', 'val', 'test']
        
    if use_case == 'train':
        samples_per_epoch = number_to_train
        shuffle_index = np.arange(0, number_to_train)
    if use_case == 'val':
        samples_per_epoch = number_to_val
        shuffle_index = np.arange(number_to_train, number_to_train + number_to_val)
    if use_case == 'test':
        samples_per_epoch = number_to_test
        shuffle_index = np.arange(number_to_train + number_to_val, number_to_train + number_to_val + number_to_test)
    
    
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    
    
    with open(file_name) as f_in:
        l_l_js = f_in.readlines()

        while 1:
            index_batch = shuffle_index[batch_size*counter:batch_size*(counter+1)]
            
            #print "counter : ", counter

            for i,j in enumerate(index_batch):

                l_js = l_l_js[j].split("\t")
                
                data, indices, indptr, pvr_z_enc = l_js[0], l_js[1], l_js[2], l_js[3]
                #print indices
                indices = np.array(json.loads(indices));  indptr  = np.array(json.loads(indptr));  data    = np.array(json.loads(data)) 
                pvr_z_enc = np.array(json.loads(pvr_z_enc)) 
                sparse2 = sparse.csr_matrix((data, indices, indptr), shape=(shape_3D[0]*shape_3D[1], shape_3D[2]))
                dense2  = np.array(sparse2.todense()).reshape(shape_3D[0], shape_3D[1], shape_3D[2],1)

                if i == 0:
                    features_binned = np.array([dense2])
                    results_enc = np.array([pvr_z_enc])
                else:
                    features_binned = np.append(features_binned, [dense2], axis=0)
                    results_enc = np.append(results_enc, [pvr_z_enc])


            
            features_binned = features_binned.reshape(batch_size,shape_3D[0], shape_3D[1], shape_3D[2],1) 
            counter += 1
            yield(features_binned,results_enc)
            if (counter > number_of_batches):
                np.random.shuffle(shuffle_index)
                counter=0

In [5]:
reader = batch_generator_readlines(abatch_size)
#reader = batch_generator_getline(abatch_size)
#reader = batch_generator_getline_backup(abatch_size)

print reader.next()
print "hi"

(array([[[[[ 0.],
          [ 0.],
          [ 0.],
          ..., 
          [ 0.],
          [ 0.],
          [ 0.]],

         [[ 0.],
          [ 0.],
          [ 0.],
          ..., 
          [ 0.],
          [ 0.],
          [ 0.]],

         [[ 0.],
          [ 0.],
          [ 0.],
          ..., 
          [ 0.],
          [ 0.],
          [ 0.]],

         ..., 
         [[ 0.],
          [ 0.],
          [ 0.],
          ..., 
          [ 0.],
          [ 0.],
          [ 0.]],

         [[ 0.],
          [ 0.],
          [ 0.],
          ..., 
          [ 0.],
          [ 0.],
          [ 0.]],

         [[ 0.],
          [ 0.],
          [ 0.],
          ..., 
          [ 0.],
          [ 0.],
          [ 0.]]],


        [[[ 0.],
          [ 0.],
          [ 0.],
          ..., 
          [ 0.],
          [ 0.],
          [ 0.]],

         [[ 0.],
          [ 0.],
          [ 0.],
          ..., 
          [ 0.],
          [ 0.],
          [ 0.]],

         [[ 0.],
     

In [ ]:
from keras.layers import Input, Dense, Dropout, Flatten 
from keras.layers import Conv3D, MaxPooling3D
from keras.models import Model



# This returns a tensor
inputs = Input(shape=(shape_3D[0],shape_3D[1],shape_3D[2],1,)) 

# 50 50 100
# 46 46 94
# 23 23 47
# 20 20 40
# 15 15 34
# 15 15 17
# 10 10 12
# 5   5  6

x = Conv3D(4, (5, 5, 7), activation='relu')(inputs) 
x = MaxPooling3D(pool_size=(2,2,2))(x)
x = Conv3D(4, (4, 4, 8), activation='relu')(inputs)
x = Conv3D(4, (6, 6, 7), activation='relu')(inputs)
x = MaxPooling3D(pool_size=(1,1,2))(x)
x = Conv3D(8, (6, 6, 6), activation='relu')(inputs)
x = MaxPooling3D(pool_size=(2,2,2))(x)
x = Flatten()(x)




# a layer instance is callable on a tensor, and returns a tensor
x = Dense(512, activation='relu')(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(1, activation='tanh')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='mse',   #mean squared error
              metrics=[])


#model.fit_generator(generator=batch_generator_readlines(batch_size=abatch_size), steps_per_epoch=number_to_train/abatch_size, epochs=5)
model.fit_generator(generator=batch_generator_readlines(batch_size=abatch_size, use_case='train'), validation_data=batch_generator_readlines(batch_size=abatch_size, use_case='val'),
        validation_steps=number_to_val // abatch_size, steps_per_epoch=number_to_train//abatch_size, epochs=5)
#model.fit_generator(generator=batch_generator_getline(batch_size=abatch_size, use_case='train'), validation_data=batch_generator_getline(batch_size=abatch_size, use_case='val'),
#        validation_steps=number_to_val // abatch_size, steps_per_epoch=number_to_train//abatch_size, epochs=5)



#model.fit(features_binned[:number_to_train,:,:,:,:], results_enc[:number_to_train], epochs=5, batch_size=25)  # starts training

Using Theano backend.


Epoch 1/5
320/320 [==============================] - 23258s - loss: 0.2094 - val_loss: 0.0092
Epoch 2/5
320/320 [==============================] - 90912s - loss: 0.0052 - val_loss: 0.0039
Epoch 3/5
  5/320 [..............................] - ETA: 15848s - loss: 0.0030

In [ ]:
score = model.evaluate_generator(generator=batch_generator_readlines(batch_size=abatch_size, use_case='test'), steps=number_to_test//abatch_size)
print score

In [ ]:
test_gen = batch_generator_readlines(batch_size=abatch_size, use_case='test')
counter = 0
max_test_steps = np.inf
for X, y in test_gen:
    if (counter >= min (number_to_test//abatch_size, max_test_steps) ): break
    counter += 1
    predicted = model.predict(X)
    comp = np.c_[y.reshape(-1,1), predicted]
    print comp